In [1]:
import paddle.fluid as fluid

In [2]:
#定义两个张量
x1 = fluid.layers.fill_constant(shape=[2,2],value=1, dtype='int64')
x2 = fluid.layers.fill_constant(shape=[2,2],value=1, dtype='int64')

In [3]:
#求和
y1 = fluid.layers.sum(x=[x1,x2])

In [4]:
#创建一个 CPU 解释器
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
#参数初始化
exe.run(fluid.default_startup_program())

/usr/local/lib/python3.7/site-packages/paddle/fluid/executor.py:804: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)


[]

In [5]:
result = exe.run(program=fluid.default_main_program(),fetch_list=[y1])#执行计算时的 program 调用的是主程序,不是上文的初始化的参数,fetch_list 存放输出的值
print(result)

[array([[2, 2],
       [2, 2]], dtype=int64)]


## 定义两个 tensor,但不赋值

In [6]:
import numpy as np

In [7]:
#定义两个张量
a = fluid.layers.create_tensor(dtype='int64',name='a')
b = fluid.layers.create_tensor(dtype='int64',name='b')

In [8]:
#求和
y = fluid.layers.sum(x=[a,b])

In [9]:
#创建一个使用 CPU 的解释器
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)
#参数初始化
exe.run(fluid.default_startup_program())

[]

In [10]:
#定义要计算的张量值
a1 = np.array([3,2]).astype('int64')
b1 = np.array([1,1]).astype('int64')

In [11]:
#运算,输出
out_a,out_b,result = exe.run(program=fluid.default_main_program(),feed={'a':a1,'b':b1},fetch_list=[a,b,y])
print(out_a," + ", out_b,' = ', result)

[3 2]  +  [1 1]  =  [4 3]


## LR with paddlepaddle

In [12]:
import paddle
import paddle.fluid as fluid
import numpy as np

In [13]:
# 定义一个简单的线性网络
x = fluid.layers.data(name='x',shape=[13],dtype='float32') #只定义形状,后期传入数据
hidden = fluid.layers.fc(input=x,size=100,act='relu')#隐藏层
net = fluid.layers.fc(input=hidden,size=1,act=None)#输出层

In [14]:
#定义 Loss函数
y = fluid.layers.data(name='y',shape=[1],dtype='float32')#y是输出值
cost = fluid.layers.square_error_cost(input=net,label=y)#使用平方差作为 loss,对输出层应用
avg_cost = fluid.layers.mean(cost)#求均值,因为 cost 实际上是 batch 的损失函数

In [15]:
#复制一个主程序,方便后续使用
test_program = fluid.default_main_program().clone(for_test=True)

In [16]:
#优化器来一个
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
opts = optimizer.minimize(avg_cost)#选最小的那一个

In [17]:
#创建 CPU 解释器
place = fluid.CPUPlace()
exe = fluid.Executor(place)
#参数初始化
exe.run(fluid.default_startup_program())

[]

In [18]:
#定义训练和测试数据
x_data = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
                   [5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')
y_data = np.array([[3.0], [5.0], [7.0], [9.0], [11.0]]).astype('float32')
test_data = np.array([[6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]).astype('float32')

In [19]:
#训练 100 个 pass
for pass_id in range(10):
    train_cost = exe.run(program=fluid.default_main_program(),feed={'x':x_data,'y':y_data},fetch_list=[avg_cost])
    print('Pass:%d, Cost:%.5f'%(pass_id,train_cost[0]))

/usr/local/lib/python3.7/site-packages/paddle/fluid/executor.py:782: UserWarning: The following exception is not an EOF exception.
  "The following exception is not an EOF exception.")


EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > paddle::platform::GetTraceBackString<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&>(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, char const*, int)
1   paddle::operators::SumOp::GetExpectedKernelType(paddle::framework::ExecutionContext const&) const
2   paddle::framework::OperatorWithKernel::ChooseKernel(paddle::framework::RuntimeContext const&, paddle::framework::Scope const&, paddle::platform::Place const&) const
3   paddle::framework::OperatorWithKernel::RunImpl(paddle::framework::Scope const&, paddle::platform::Place const&, paddle::framework::RuntimeContext*) const
4   paddle::framework::OperatorWithKernel::RunImpl(paddle::framework::Scope const&, pa

In [21]:
#还是有问题的.

In [ ]:
#开始预测
result = exe.run(program=test_program,feed={'x':test_data,'y':np.array([[0.0]])})